In [2]:
import csv
import pandas as pd
import psycopg2


In [3]:
movieId = []
movieName = []
movieYear = []
with open('IMDBMovieTitles.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        try:
            mn, not_good_d = row[1].split('(')
            d, bad_d = not_good_d.split(')')
        except:
            continue
            #mn = row[1]
            #d = 'Year' 
        movieId.append(row[0])
        movieName.append(mn.strip())
        movieYear.append(d)

In [7]:
#Saving to CSV
df = pd.DataFrame({'id':   movieId,
                   'year': movieYear, 
                   'name': movieName})
df.to_csv('result.csv', index=False)

In [9]:
#Connection
conn = psycopg2.connect(database="lab1", user="postgres",
    password="12345", host="localhost", port=5432)

In [10]:
cur = conn.cursor()

In [11]:
#Creation of a new table
""""
cur.execute("create table movies (id integer generated by default as identity primary key, " + 
  "year integer, name text)")

conn.commit()
"""

'"\ncur.execute("create table movies (id integer generated by default as identity primary key, " + \n  "year integer, name text)")\n\nconn.commit()\n'

In [12]:
cur.close()

In [13]:
def find(text, connection):
    #newText = text
    txt = text.split()
    year = ''
    
    for word in txt:
        if word.isnumeric() and len(word) == 4:
            year = word
            break
    
    if year != '':
        text = text.replace(year, '', 1)
    
    text = text.strip()
    
    cur = connection.cursor()
    
    if year == '':
        cur.execute("Select * from movies where  name like '{}' limit 10".format(text))
    elif text == '':
        cur.execute("Select * from movies where year = '{}' limit 10".format(year))
    else:
        cur.execute("Select * from movies where  name like '{}' and year = '{}' limit 10".format(text, year))
    rows = cur.fetchall()
    res = []
    for row in rows:
        res.append('{}, {}, {}'.format(row[0], row[1], row[2]))
    
    cur.close()
    return res

In [14]:
f = find("1995", conn)
for r in f:
    print(r)

105601, 1995, Titanica
109028, 1995, ? k”ldum klaka
109370, 1995, Canadian Bacon
109415, 1995, Children of the Corn III Urban Harvest
109642, 1995, Dolores Claiborne
109723, 1995, Embrace of the Vampire
109835, 1995, Frank &amp; Jesse
110006, 1995, Heavy Weights
110066, 1995, Houseguest
110443, 1995, Major Payne
